# Gerando texto Shakesperiano com uma Char-RNN

In [1]:
%%time
import numpy as np
from tensorflow import keras
import tensorflow as tf

2023-07-30 13:25:43.206010: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 13:25:43.306669: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 13:25:43.309776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 13:25:45.183414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


CPU times: user 4.08 s, sys: 551 ms, total: 4.63 s
Wall time: 4.16 s


## Obtendo o conjunto de treinamento

Primeiramente, faremos o download de todo o trabalho de Shakespeare, usando a função `get_file()` do Keras e baixando os dados do projeto Char-RNN de Andrej Karpathy.

In [2]:
%%time
shakespeare_url = "https://homl.info/shakespeare"

filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

CPU times: user 27.8 ms, sys: 19.9 ms, total: 47.6 ms
Wall time: 45.6 ms


In [3]:
#print(shakespeare_text)

Vamos codificar cada caractere como um inteiro, utilizando a classe `Tokenizer` do Keras. Ele encontrará todos os caracteres usados no texto e mapeará cada um deles para um ID de caractere diferente, de 1 ao número de caracteres diferentes.

In [4]:
%%time
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

CPU times: user 52 µs, sys: 7 µs, total: 59 µs
Wall time: 72.5 µs


Com `char_level = True`, obtemos a codificação em nível de caractere em vez da codificação padrão por palavras.

In [5]:
%%time
tokenizer.fit_on_texts([shakespeare_text])

CPU times: user 639 ms, sys: 38.2 ms, total: 677 ms
Wall time: 675 ms


Codificando palavras:

In [6]:
%%time
sequencia = tokenizer.texts_to_sequences(['Carlos', 'Alberto', 'Silva', 'Junior'])
sequencia

CPU times: user 86 µs, sys: 12 µs, total: 98 µs
Wall time: 108 µs


[[19, 5, 9, 12, 4, 8],
 [5, 12, 22, 2, 9, 3, 4],
 [8, 6, 12, 26, 5],
 [33, 14, 10, 6, 4, 9]]

Decodificando o texto:

In [7]:
%%time
texto = tokenizer.sequences_to_texts(sequencia)
texto

CPU times: user 85 µs, sys: 0 ns, total: 85 µs
Wall time: 94.4 µs


['c a r l o s', 'a l b e r t o', 's i l v a', 'j u n i o r']

Por padrão, o tokenizer converte o texto para minúsculo, mas é possível definir `lower=False` para evitar isso.

In [8]:
%%time
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts([shakespeare_text])

CPU times: user 640 ms, sys: 0 ns, total: 640 ms
Wall time: 638 ms


In [9]:
%%time
sequencia = tokenizer.texts_to_sequences(['Carlos', 'Alberto', 'Silva', 'Junior'])
sequencia

CPU times: user 49 µs, sys: 6 µs, total: 55 µs
Wall time: 62.2 µs


[[38, 5, 8, 12, 4, 7],
 [27, 12, 23, 2, 8, 3, 4],
 [36, 10, 12, 28, 5],
 [59, 14, 9, 10, 4, 8]]

In [10]:
%%time
texto = tokenizer.sequences_to_texts(sequencia)
texto

CPU times: user 81 µs, sys: 10 µs, total: 91 µs
Wall time: 101 µs


['C a r l o s', 'A l b e r t o', 'S i l v a', 'J u n i o r']

Deixando o texto e minúsculo mesmo:

In [11]:
%%time
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

CPU times: user 887 ms, sys: 18.1 ms, total: 905 ms
Wall time: 898 ms


Verificando o número de caracteres:

In [12]:
%%time
max_id = len(tokenizer.word_index)
max_id

CPU times: user 36 µs, sys: 4 µs, total: 40 µs
Wall time: 47.9 µs


39

Vamos codificar o texto de Shakespeare completamente:

In [13]:
%%time
#Temos ids de 1 a 39, mas subtraímos 1 para deixar de 0 a 38
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

CPU times: user 782 ms, sys: 50 ms, total: 832 ms
Wall time: 830 ms


In [14]:
encoded

array([19,  5,  8, ..., 20, 26, 10])

Verificando o total de caracteres:

In [15]:
%%time
'''
Obs.: No livro, refere-se a função tokenizer.document_count para obter o total de caracteres.
Porém, essa função retorna o número de documentos utilizados para treinar o tokenizer. No
nosso caso sempre retorna 1
'''
#dataset_size = tokenizer.document_count
dataset_size = len(encoded)
dataset_size

CPU times: user 48 µs, sys: 5 µs, total: 53 µs
Wall time: 63.2 µs


1115394

## Separando em conjunto de treino e teste

Vamos dividir os primeiros 90% do texto para o conjunto de treinamento.

In [16]:
%%time
train_size = dataset_size * 90 // 100 #// retorna apenas a parte inteira da divisão
train_size

CPU times: user 54 µs, sys: 6 µs, total: 60 µs
Wall time: 71.8 µs


1003854

In [17]:
%%time
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset

CPU times: user 55.3 ms, sys: 37.2 ms, total: 92.5 ms
Wall time: 86 ms


<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

O que houve acima?

`tf.data.Dataset.from_tensor_slices()`: Esta é uma função do TensorFlow que cria um objeto de conjunto de dados (Dataset) a partir de um tensor (ou sequência) ou de um conjunto de tensores. Neste caso, estamos usando o tensor resultante do slicing `encoded[:train_size]` como entrada para criar o conjunto de dados.

Portanto, esse trecho de código cria um objeto de conjunto de dados que contém os primeiros train_size elementos do tensor encoded. Esse conjunto de dados pode ser usado em operações de treinamento ou avaliação de modelos no TensorFlow. Mais [aqui](https://www.geeksforgeeks.org/tensorflow-tf-data-dataset-from_tensor_slices/).

## Decompondo o conjunto de dados sequencial em janelas

Vamos utilizar o método window() para criar um conjunto de dados de janelas de textos curto. Cada instância no conjunto de dados será uma sequência de caracteres curta de todo o texto, e a RNN será desenrolada ao longo dessas subsequências de caracteres. Isso se chama `retropropagação truncada ao longo do tempo`.

In [18]:
%%time
n_steps = 100
window_len = n_steps + 1
dataset = dataset.window(window_len, shift=1, drop_remainder=True)

CPU times: user 2.7 ms, sys: 29.2 ms, total: 31.9 ms
Wall time: 30.4 ms


O que houve acima?

O método `window` cria um conjunto de dados que contem janelas, cada uma representada como um conjunto de dados. É um conjunto aninhado de dados, como uma lista de listas.

O método `window` cria janelas não sobrepostas, mas para obter o maior conjunto possível utilizamos `shift=1`, de modo que a primeira janela contém caracteres de 0 a 100, a segunda de 1 a 101, e assim por diante.

Para assegurar que todas as janelas contenham 101 caracteres, utilizamos `drop_remainder=True`, já que nas janelas finais teríamos janelas com número de caracteres cada vez menor.  

Agora vamos tratar as janelas para que possam ser utilizadas no modelo, convertendo nosso conjunto aninhado de dados em um `conjunto de dados flat`.

In [19]:
%%time
dataset = dataset.flat_map(lambda window: window.batch(window_len))

Cause: could not parse the source code of <function <lambda> at 0x7fcb2a9535b0>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fcb2a9535b0>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
CPU times: user 65.2 ms, sys: 9.18 ms, total: 74.4 ms
Wall time: 70.6 ms


O que houve acima?


`flat_map`: O método flat_map é uma operação de transformação aplicada em um objeto Dataset. Essa operação mapeia cada elemento do conjunto de dados original para um novo conjunto de elementos (que pode ter um número diferente de elementos em relação ao original) e, em seguida, "aplanará" ou "achata" o resultado, combinando todos os elementos em um único conjunto de dados. Em outras palavras, ele concatena as saídas dos mapeamentos individuais em um único conjunto de dados.

`lambda window: window.batch(window_len)`: Esta é uma expressão lambda (função anônima) que está sendo passada como argumento para o flat_map. A expressão lambda recebe um argumento chamado window e aplica uma operação em window usando o método batch(window_len).

`window`: Cada elemento do conjunto de dados original é uma "janela" de dados de tamanho fixo. Pode-se imaginar um conjunto de séries temporais, em que cada elemento representa uma janela de window_len pontos de dados.

`window.batch(window_len)`: Essa parte do código usa o método batch para agrupar os elementos da janela em lotes de tamanho fixo de window_len. O método batch é usado para criar lotes a partir do conjunto de dados, e aqui está sendo usado para agrupar cada "janela" em um lote de window_len elementos.

O resultado disso é que o código cria lotes de tamanho window_len a partir de cada "janela" do conjunto de dados original e, em seguida, a função flat_map concatena todos esses lotes em um único conjunto de dados. Essa operação pode ser útil em tarefas de aprendizado de máquina, especialmente quando se trabalha com séries temporais ou outros tipos de dados estruturados em janelas fixas.

Vamos embaralhar as janelas, agrupá-las e separar o último caractere de cada janela, já que este é o nosso alvo. Lembrando que o objetivo do modelo será prever o último caractere de uma sequência fornecida.

In [ ]:
%%time
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:,-1:]))

#Versão do livro: dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:,1:]))
''' Apresenta um erro no slice para obter o último caractere. Deve ser windows[:,-1:] para \
obter o último caractere do conjunto '''

O que houve acima?

`map`: O método map é uma operação de transformação aplicada em um objeto Dataset. Essa operação mapeia cada elemento do conjunto de dados original para um novo elemento, de acordo com uma função especificada. A saída do map terá o mesmo número de elementos do conjunto de dados original, mas os elementos terão sido transformados de acordo com a função fornecida.

`lambda windows: (windows[:, :-1], windows[:,-1:])`: Esta é uma expressão lambda (função anônima) que está sendo passada como argumento para o map. A expressão lambda recebe um argumento chamado windows, que representa um elemento do conjunto de dados original (uma "janela" de dados de tamanho fixo).

`(windows[:, :-1], windows[:,-1:])`: Essa parte da expressão lambda aplica a operação de divisão na janela de dados. A janela é dividida em duas partes:

* `windows[:, :-1]`: Seleciona todas as linhas da janela e todas as colunas, exceto a última coluna. Ou seja, estamos selecionando todas as colunas, exceto a última. Essa parte corresponde a todas as características (ou entradas) da janela, exceto o último elemento.

* `windows[:, -1:]`: Seleciona todas as linhas da janela e apenas a última coluna. Essa parte corresponde ao último elemento da janela, que é considerado como a saída (ou rótulo) associada às características.

Portanto, a função lambda está transformando cada janela de dados em um par ordenado, onde o primeiro elemento do par são todas as características (exceto o último elemento) e o segundo elemento do par é o último elemento (que é a saída desejada) da janela. Isso é frequentemente usado em tarefas de aprendizado de máquina supervisionado, onde você deseja separar as entradas dos rótulos (saídas) para alimentar um modelo de aprendizado de máquina com os dados corretos.

Vamos codificar cada caractere usando um vetor one-hot, porque existem apenas 39 caracteres diferentes.

In [ ]:
%%time
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

Por fim, vamos adicionar a pré-busca

In [22]:
dataset = dataset.prefetch(1)

O que houve acima?

A função prefetch é uma das várias transformações que podem ser aplicadas a um objeto dataset. Sua finalidade é melhorar a eficiência do carregamento dos dados, permitindo que o pipeline de entrada pré-carregue um lote (ou mais) de dados enquanto o modelo está ocupado processando o lote anterior.

Quando chamamos `ataset.prefetch(1)`, estamos instruindo o TensorFlow a pré-carregar um único lote de dados, mantendo-o em um buffer de memória. Isso significa que, quando o modelo estiver prestes a terminar de processar o lote atual, o próximo lote já estará disponível na memória para ser acessado imediatamente. Isso ajuda a evitar gargalos de desempenho relacionados ao carregamento de dados e permite que a CPU/GPU seja mais eficientemente utilizada.

O argumento `(1)` indica que apenas um lote será pré-carregado. Você pode ajustar esse valor com base nas necessidades do seu modelo e da capacidade de memória disponível. Em geral, é uma boa prática manter o valor pequeno para não ocupar muita memória e evitar atrasos desnecessários, mas isso pode variar dependendo do cenário.

Em resumo, a linha de código `dataset = dataset.prefetch(1)` está configurando o objeto dataset para pré-carregar um único lote de dados, melhorando potencialmente a eficiência de carregamento e processamento de dados durante o treinamento do modelo. Isso é especialmente útil quando a carga de dados é uma parte significativa do tempo de treinamento e pode ajudar a acelerar o treinamento em geral.